In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as mdb
import ujson as js
import torch as pt
import tqdm
import mysql.connector as mdb

sns.set()

from utils.market_parser import market_smooth
from utils.sql_pullers import get_types

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def mat_eff_calc(mat_quant, me_level):
    quant_actual = np.maximum(1, np.ceil(np.round(mat_quant * ((100 - me_level) / 100), 2))).astype(int)
    return quant_actual

In [3]:
with open('./utils/sql_calls.json') as file:
    sql = js.load(file)

mat_region_id = 10000043
sale_region_id = 10000052
#sale_region_id = 10000043

h = 14
markets = {}
prices = {}

market_start = '2019-01-01'

In [4]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

In [5]:
corp_bps = pd.read_sql(sql['corp']['bpos'], conn, index_col='bp_item_id')
corp_bps = corp_bps.loc[corp_bps[['bp_type_id', 'mat_eff', 'time_eff']].drop_duplicates().index]
corp_bps.index = corp_bps.index.astype(str)

In [6]:
corp_orders = pd.read_sql(sql['corp']['orders'], conn, index_col='order_id')

In [7]:
bp_data = {}
for key, script in sql['bp'].items():
    bp_data[key] = pd.read_sql(script.format(type_ids=','.join(corp_bps['bp_type_id'].astype(str))), conn)

In [8]:
invent = {
    'mats': {
        'quant': corp_bps.reset_index().merge(
            bp_data['mats'].loc[bp_data['mats']['act_id'] == 8],
            on='bp_type_id'
        )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']].pivot(
            index='bp_item_id',
            columns='mat_type_id',
            values='mat_quant'
        ).fillna(0)
    }
}

In [9]:
markets['invent'] = market_smooth(mat_region_id, invent['mats']['quant'].columns, h, start_date=market_start)

prices[mat_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['invent'][mat_region_id].items()]
).set_index('type_id')

invent['mats']['price'] = pd.concat([
    (prices[mat_region_id]['avg_price'] * invent['mats']['quant']).sum(axis=1),
    (prices[mat_region_id]['variance'] * (invent['mats']['quant']**2)).sum(axis=1)
], axis=1)
invent['mats']['price'].columns = prices[mat_region_id].columns

/home/calvinxc1/conda/envs/analysis/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
invent['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 8],
    on='bp_type_id'
)[['bp_item_id', 'bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
invent['prods']['avg_price_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['avg_price'])
invent['prods']['variance_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['variance'])
invent['prods']['avg_price_unit'] = invent['prods']['avg_price_run'] / invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)
invent['prods']['variance_unit'] = invent['prods']['variance_run'] * (invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)**2)
invent['prods'].index = invent['prods']['bp_item_id'] + ':' + invent['prods']['prod_type_id'].astype(str)
invent['prods'].index.name = 'bp_item_id'
invent['prods'].drop(['bp_item_id', 'bp_type_id'], axis=1, inplace = True)

In [11]:
invent_bps = corp_bps.loc[invent['prods'].index.to_series().apply(lambda x: x.split(':')[0])]
invent_bps.index = invent['prods'].index
invent_bps['bp_type_id'] = invent['prods']['prod_type_id']
invent_bps['mat_eff'] = 2
invent_bps['time_eff'] = 4
corp_bps = corp_bps.append(invent_bps)
corp_bps.index.name = 'bp_item_id'
del invent_bps

In [12]:
for key, sql in sql['bp'].items():
    bp_data[key] = bp_data[key].append(
        pd.read_sql(sql.format(type_ids=','.join(invent['prods']['prod_type_id'].unique().astype(str))), conn),
        ignore_index=True
    ).drop_duplicates().reset_index(drop=True)

In [13]:
build = {
    'count': corp_bps.reset_index().merge(
        bp_data['mats'].loc[bp_data['mats']['act_id'] == 1],
        on='bp_type_id'
    )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']]
}
build['count']['mat_actual'] = np.maximum(1, np.ceil(np.round(build['count']['mat_quant'] * ((100 - build['count']['mat_eff']) / 100), 2)))
build['count'] = build['count'].pivot(index='bp_item_id', columns='mat_type_id', values='mat_actual').fillna(0)

In [14]:
sub_prods = bp_data['prods'].loc[
    (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')

active = True
while active:
    build_mod = build['count'][sub_prods.index] @ sub_prods
    build['count'][sub_prods.index] = 0
    build['count'].loc[build_mod.index, build_mod.columns] += build_mod
    build['count'] = build['count'].loc[:, build['count'].sum(axis=0) > 0]
    
    sub_prods = bp_data['prods'].loc[
        (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
    ][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
    sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
    sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
    sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
    sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
    sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
    sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
    sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')
    
    if len(sub_prods) == 0: active = False

In [15]:
markets['build'] = market_smooth(mat_region_id, build['count'].columns, h, start_date=market_start)

prices[mat_region_id] = prices[mat_region_id].append(pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['build'][mat_region_id].items()]
).set_index('type_id')).drop_duplicates().sort_index()

In [16]:
build['price'] = pd.concat([
    (build['count'] * prices[mat_region_id]['avg_price']).sum(axis=1),
    ((build['count']**2) * prices[mat_region_id]['variance']).sum(axis=1)
], axis=1)
build['price'].columns = ['%s_run' % col for col in prices[mat_region_id].columns]

In [17]:
build['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 1],
    on='bp_type_id'
)[['bp_item_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
build['prods']['avg_price_run'] = build['prods']['bp_item_id'].map(build['price']['avg_price_run'])
build['prods']['variance_run'] = build['prods']['bp_item_id'].map(build['price']['variance_run'])
build['prods'].set_index('bp_item_id', inplace=True)

In [18]:
build['cost'] = build['prods'][['prod_type_id', 'prod_quant', 'prod_prob', 'avg_price_run', 'variance_run']].join(
    invent['prods'][['avg_price_unit', 'variance_unit']]
).fillna(0)
build['cost'].rename(columns={'avg_price_unit':'avg_price_invent', 'variance_unit':'variance_invent'}, inplace=True)

In [19]:
build['cost']['avg_price_unit'] = (
    build['cost']['avg_price_run'] + build['cost']['avg_price_invent']
) / (
    build['cost']['prod_quant'] * build['cost']['prod_prob']
)
build['cost']['variance_unit'] = (
    build['cost']['variance_run'] + build['cost']['variance_invent']
) / (
    (build['cost']['prod_quant'] * build['cost']['prod_prob'])**2
)

In [20]:
markets['sale'] = market_smooth(sale_region_id, build['cost']['prod_type_id'].unique(), h, start_date=market_start)

prices[sale_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

In [21]:
build['cost']['avg_price_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['avg_price'])
build['cost']['variance_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['variance'])
build['cost']['avg_profit'] = build['cost']['avg_price_sale'] - build['cost']['avg_price_unit']
build['cost']['profit_ratio'] = build['cost']['avg_profit'] / build['cost']['avg_price_unit']
build['cost']['variance_profit'] = build['cost']['variance_sale'] + build['cost']['variance_unit']
build['cost']['stdev_profit'] = np.sqrt(build['cost']['variance_profit'])
build['cost']['z_profit'] = build['cost']['avg_profit'] / (np.sqrt(build['cost']['variance_sale']) + np.sqrt(build['cost']['variance_unit']))

In [22]:
types = get_types(build['cost']['prod_type_id'])
build['cost']['prod_type_name'] = build['cost']['prod_type_id'].map(types['type_name'])
build['cost']['prod_group_name'] = build['cost']['prod_type_id'].map(types['group_name'])
build['cost']['prod_cat_name'] = build['cost']['prod_type_id'].map(types['category_name'])

In [23]:
volume = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_volume': vals[h]['level'].iloc[-1]['volume'],
        'variance': vals[h]['var'].iloc[-1]['volume']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

build['cost']['volume'] = build['cost']['prod_type_id'].map(volume['avg_volume'])
build['cost']['vol_var'] = build['cost']['prod_type_id'].map(volume['variance'])
build['cost']['avg_value'] = build['cost']['avg_profit'] * build['cost']['volume']
build['cost']['var_value'] = (
    build['cost']['variance_profit'] * build['cost']['vol_var']
) + (
    build['cost']['variance_profit'] * (build['cost']['volume']**2)
) + (
    (build['cost']['avg_profit']**2) * build['cost']['vol_var']
)
build['cost']['stdev_value'] = np.sqrt(build['cost']['var_value'])

In [24]:
build['cost']['active_order'] = build['cost']['prod_type_id'].isin(corp_orders['type_id'])

In [29]:
build['cost'].loc[
    ~build['cost']['active_order'],
    [
        'prod_cat_name', 'prod_group_name', 'prod_type_name', 'avg_price_unit',
        'avg_price_sale', 'avg_profit', 'profit_ratio', 'volume', 'avg_value'
    ]
].sort_values('avg_value', ascending=False).iloc[0:50]

,prod_cat_name,prod_group_name,prod_type_name,avg_price_unit,avg_price_sale,avg_profit,profit_ratio,volume,avg_value
bp_item_id,,,,,,,,,
278129470,Ship,Battleship,Armageddon,"129,977,734.83","155,706,407.12","25,728,672.29",0.20,0.23,"5,976,531.35"
470571935:12817,Charge,Advanced Pulse Laser Crystal,Conflagration L,"242,231.97","659,259.54","417,027.56",1.72,10.77,"4,490,866.89"
1010832340145:33698,Ship,Expedition Frigate,Prospect,"22,771,690.64","36,696,322.16","13,924,631.53",0.61,0.14,"1,891,289.48"
443581437:30837,Module,Salvager,Salvager II,"939,290.54","1,595,939.66","656,649.12",0.70,2.60,"1,708,885.91"
1532766410:21639,Drone,Combat Drone,Vespa II,"725,704.05","1,031,502.51","305,798.46",0.42,4.72,"1,444,469.19"
1051212744:2437,Drone,Combat Drone,Wasp II,"1,424,959.35","2,250,645.50","825,686.15",0.58,1.36,"1,125,364.57"
436324072:2532,Module,Shield Resistance Amplifier,Explosive Deflection Amplifier II,"687,017.78","2,634,270.83","1,947,253.06",2.83,0.52,"1,020,591.79"
1051213588:2196,Drone,Combat Drone,Praetor II,"1,448,094.04","2,199,352.45","751,258.41",0.52,1.35,"1,015,749.09"
436309419:1249,Module,Capacitor Flux Coil,Capacitor Flux Coil II,"572,642.85","2,257,362.80","1,684,719.95",2.94,0.59,"998,115.18"


In [26]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

with open('./utils/sql.json') as file:
    undercuts = pd.read_sql(js.load(file)['undercuts'], conn, index_col = 'order_id')
undercuts['market_volume_sum'] = undercuts['market_volume_sum'].astype(int)
types = types.append(get_types([type_id for type_id in undercuts['type_id'].unique() if type_id not in types.index])).sort_index()
undercuts.insert(
    undercuts.columns.get_loc('type_id') + 1,
    'type_name',
    undercuts['type_id'].map(types['type_name'])
)

conn.close()

thresh = 0.1

mask = undercuts['volume'] * thresh <= undercuts['market_volume_sum']

In [27]:
print(undercuts.loc[mask].shape)
undercuts.loc[mask].sort_values('type_name', ascending=True).iloc[:50]

(8, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5506972257,442,Cargo Scanner I,10000052,"27,500.00",100,1,"25,000.00",12,12
5482099175,18048,Hemorphite Mining Crystal I,10000052,"60,000.00",96,1,"54,000.00",10,10
5504398650,3528,Medium Armor Repairer I,10000052,"125,000.00",100,1,"114,000.00",10,10
5482099208,18062,Scordite Mining Crystal I,10000052,"13,500.00",173,1,"12,000.00",21,21
5506957612,1185,Small Cap Battery I,10000052,"70,000.00",93,1,"63,000.00",30,30
5503605318,444,Survey Scanner I,10000052,"32,500.00",99,1,"30,000.00",30,30
5482099230,18066,Veldspar Mining Crystal I,10000052,"8,000.00",135,1,"7,200.00",40,40
5506957615,252,Xray M,10000052,"10,000.00",100,1,"9,000.00",10,10


In [28]:
print(undercuts.loc[~mask].shape)
undercuts.loc[~mask].sort_values('type_name', ascending=True).iloc[:50]

(72, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5483836089,12344,200mm Railgun I,10000052,"95,000.00",100,1,"86,000.00",4,4
5504398645,493,720mm Howitzer Artillery I,10000052,"400,000.00",100,1,"360,000.00",1,1
5483836091,561,75mm Gatling Rail I,10000052,"17,500.00",92,1,"2,000.00",3,3
5511972392,2281,Adaptive Invulnerability Field II,10000052,"2,750,000.00",96,1,"2,700,000.00",1,1
5482099146,42832,Armor Energizing Charge,10000052,600.00,6900,1,540.00,470,470
5503605315,608,Atron,10000052,"245,000.00",25,1,"200,000.00",1,1
5482099152,3554,Cap Booster 100,10000052,750.00,4652,1,680.00,50,50
5482099153,11285,Cap Booster 200,10000052,"21,750.00",2263,1,"19,000.00",152,152
5513937085,2032,Cap Recharger II,10000052,"875,000.00",99,1,"870,000.00",1,1
